In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [11]:
url_inicial='https://www.revistascca.unam.mx/atm/index.php/atm/issue/view/36-1'
url_root= 'https://www.revistascca.unam.mx/atm/index.php/atm/issue/view/36-1'
r = requests.get(url_inicial)

In [12]:
soup = BeautifulSoup(r.text, 'html.parser')

In [13]:
box = soup.find('div', class_='media-list')
volumen=box.findAll('div', class_='article-summary media')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.revistascca.unam.mx/atm/index.php/atm/article/view/52953',
 'https://www.revistascca.unam.mx/atm/index.php/atm/article/view/53017',
 'https://www.revistascca.unam.mx/atm/index.php/atm/article/view/52858',
 'https://www.revistascca.unam.mx/atm/index.php/atm/article/view/53025',
 'https://www.revistascca.unam.mx/atm/index.php/atm/article/view/53028',
 'https://www.revistascca.unam.mx/atm/index.php/atm/article/view/53026',
 'https://www.revistascca.unam.mx/atm/index.php/atm/article/view/53002',
 'https://www.revistascca.unam.mx/atm/index.php/atm/article/view/53027',
 'https://www.revistascca.unam.mx/atm/index.php/atm/article/view/52940',
 'https://www.revistascca.unam.mx/atm/index.php/atm/article/view/52966']

In [14]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='media-list')
    volumen=box.findAll('div', class_='article-summary media')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [15]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

Estoy en la pagina https://www.revistascca.unam.mx/atm/index.php/atm/issue/view/36-1


In [16]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

10

In [17]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [43]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Atmósfera'
    a='I. Físico Matemáticas y Ciencias de la Tierra'
    tem='Ciencias Planetarias y de la Atmósfera'
    url='http://www.revistascca.unam.mx/atm'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #abstract
    try:
        abst=s_item.find('p').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #resumen
    try:
        resu=s_item.find('div', class_='journal-logo div_imagen_banner').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', role='button').get('data-href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [44]:
list_scraper=list_scraper[0:10]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9


In [45]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Abstract,Resumen,Link inicial,Link articulo
0,Atmósfera,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Planetarias y de la Atmósfera,Classification of the flood severity of the Gu...,This study estimates the flood severity betwee...,None,http://www.revistascca.unam.mx/atm,https://www.revistascca.unam.mx/atm/index.php/...
1,Atmósfera,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Planetarias y de la Atmósfera,Behavior of the ITCZ second band near the Peru...,The behavior of the second band of the Intertr...,None,http://www.revistascca.unam.mx/atm,https://www.revistascca.unam.mx/atm/index.php/...
2,Atmósfera,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Planetarias y de la Atmósfera,Surface shortwave cloud radiative effect of cu...,The effects of cumulus (Cu) clouds and the com...,None,http://www.revistascca.unam.mx/atm,https://www.revistascca.unam.mx/atm/index.php/...
3,Atmósfera,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Planetarias y de la Atmósfera,Summer dry events on synoptic and intraseasona...,The occurrence of dry events in the Southeast ...,None,http://www.revistascca.unam.mx/atm,https://www.revistascca.unam.mx/atm/index.php/...
4,Atmósfera,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Planetarias y de la Atmósfera,Connecting heavy precipitation events to outgo...,Spatial fields of outgoing long wave radiation...,None,http://www.revistascca.unam.mx/atm,https://www.revistascca.unam.mx/atm/index.php/...
5,Atmósfera,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Planetarias y de la Atmósfera,"Regional patterns of vegetation, temperature, ...","Changes in atmospheric CO2, ocean temperature,...",None,http://www.revistascca.unam.mx/atm,https://www.revistascca.unam.mx/atm/index.php/...
6,Atmósfera,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Planetarias y de la Atmósfera,"Agroclimatic zoning of the state of Nayarit, M...",Agriculture productivity in the state of Nayar...,None,http://www.revistascca.unam.mx/atm,https://www.revistascca.unam.mx/atm/index.php/...
7,Atmósfera,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Planetarias y de la Atmósfera,Partial COVID-19 lockdown effect in atmospheri...,The COVID-19 pandemic introduced a significant...,None,http://www.revistascca.unam.mx/atm,https://www.revistascca.unam.mx/atm/index.php/...
8,Atmósfera,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Planetarias y de la Atmósfera,Quantification of greenhouse gas emissions of ...,This work aimed to identify and quantify green...,None,http://www.revistascca.unam.mx/atm,https://www.revistascca.unam.mx/atm/index.php/...
9,Atmósfera,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Planetarias y de la Atmósfera,Prediction of atmospheric corrosion from meteo...,The assessment of atmospheric corrosion is cur...,None,http://www.revistascca.unam.mx/atm,https://www.revistascca.unam.mx/atm/index.php/...


In [46]:
df_catalogo.to_csv('Revista13.01.csv', index=False)